In [80]:
%reload_ext autoreload
%autoreload 2

from utils import load_dataset, SpecialTokens
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import numpy as np
import lightning.pytorch as pl

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [81]:
train_size = 10000
val_size = 1000
train_data, human_vocab, machine_vocab = load_dataset(train_size)
val_data, _, _ = load_dataset(val_size)

100%|██████████| 1000/1000 [00:00<00:00, 33722.23it/s]


In [82]:
human_vocab

{';': 0,
 '?': 1,
 ' ': 2,
 '.': 3,
 '/': 4,
 '0': 5,
 '1': 6,
 '2': 7,
 '3': 8,
 '4': 9,
 '5': 10,
 '6': 11,
 '7': 12,
 '8': 13,
 '9': 14,
 'a': 15,
 'b': 16,
 'c': 17,
 'd': 18,
 'e': 19,
 'f': 20,
 'g': 21,
 'h': 22,
 'i': 23,
 'j': 24,
 'l': 25,
 'm': 26,
 'n': 27,
 'o': 28,
 'p': 29,
 'r': 30,
 's': 31,
 't': 32,
 'u': 33,
 'v': 34,
 'w': 35,
 'y': 36}

In [83]:
machine_vocab

{';': 0,
 '>': 1,
 '<': 2,
 '-': 3,
 '0': 4,
 '1': 5,
 '2': 6,
 '3': 7,
 '4': 8,
 '5': 9,
 '6': 10,
 '7': 11,
 '8': 12,
 '9': 13}

In [84]:
train_data[:20]

[('5/8/10', '>2010-05-08<'),
 ('9/3/72', '>1972-09-03<'),
 ('5/4/80', '>1980-05-04<'),
 ('3/7/85', '>1985-03-07<'),
 ('8/4/80', '>1980-08-04<'),
 ('5/4/00', '>2000-05-04<'),
 ('5/4/07', '>2007-05-04<'),
 ('5/1/78', '>1978-05-01<'),
 ('2/1/72', '>1972-02-01<'),
 ('3/6/07', '>2007-03-06<'),
 ('8/7/11', '>2011-08-07<'),
 ('3/7/14', '>2014-03-07<'),
 ('2/3/85', '>1985-02-03<'),
 ('5/7/83', '>1983-05-07<'),
 ('6/6/92', '>1992-06-06<'),
 ('3/4/94', '>1994-03-04<'),
 ('7/3/74', '>1974-07-03<'),
 ('1/5/78', '>1978-01-05<'),
 ('1/5/79', '>1979-01-05<'),
 ('9/5/07', '>2007-09-05<')]

In [85]:
val_data[:20]

[('2/8/23', '>2023-02-08<'),
 ('4/9/07', '>2007-04-09<'),
 ('5/7/99', '>1999-05-07<'),
 ('2/9/04', '>2004-02-09<'),
 ('4/2/92', '>1992-04-02<'),
 ('12/5/99', '>1999-12-05<'),
 ('8/18/77', '>1977-08-18<'),
 ('7 06 16', '>2016-06-07<'),
 ('7/23/17', '>2017-07-23<'),
 ('7/15/84', '>1984-07-15<'),
 ('3/19/86', '>1986-03-19<'),
 ('4 01 21', '>2021-01-04<'),
 ('8 02 02', '>2002-02-08<'),
 ('4/26/94', '>1994-04-26<'),
 ('7/14/18', '>2018-07-14<'),
 ('5/25/81', '>1981-05-25<'),
 ('1/18/80', '>1980-01-18<'),
 ('8/13/77', '>1977-08-13<'),
 ('5 10 16', '>2016-10-05<'),
 ('4/12/94', '>1994-04-12<')]

In [86]:
class Lang:
    def _get_char(self, ind):
        if isinstance(ind, torch.Tensor):
            return self.inv_vocab[ind.item()]
        else:
            return self.inv_vocab[ind]

    def __init__(self, vocab: dict):
        self.vocab = vocab
        self.inv_vocab = {v:k for k,v in vocab.items()}
        self.vocab_size = len(vocab)

    def str_to_ind(self, str):
        return [self.vocab[c] for c in str]
    
    def ind_to_str(self, ind):
        return ''.join([self._get_char(i) for i in ind])

In [87]:
test = Lang(human_vocab)
date = train_data[0][0]
print(date)
translated_date = test.str_to_ind(date)
print(translated_date)
reversed_translation = test.ind_to_str(translated_date)
print(reversed_translation)

5/8/10
[10, 4, 13, 4, 6, 5]
5/8/10


In [88]:
class TranslationTrainingDataset(Dataset):
    def __init__(self, data, input_vocab, output_vocab):
        self.input_lang = Lang(input_vocab)
        self.target_lang = Lang(output_vocab)

        self.data = data

        self.encoder_inputs = [self.input_lang.str_to_ind(input_sent) for input_sent, _ in self.data]

        targets = [self.target_lang.str_to_ind(target_sent) for _, target_sent in self.data]
        self.decoder_inputs = [target[:-1] for target in targets]
        self.decoder_targets = [target[1:] for target in targets]

    def __getitem__(self, index):
        return self.encoder_inputs[index], self.decoder_inputs[index], self.decoder_targets[index]
    
    def __len__(self):
        return len(self.encoder_inputs)


In [89]:
train_dataset = TranslationTrainingDataset(train_data, human_vocab, machine_vocab)
val_dataset = TranslationTrainingDataset(val_data, human_vocab, machine_vocab)

In [90]:
x,y,z = train_dataset[0]
print(x, train_dataset.input_lang.ind_to_str(x))
print(y, train_dataset.target_lang.ind_to_str(y))
print(z, train_dataset.target_lang.ind_to_str(z))

[10, 4, 13, 4, 6, 5] 5/8/10
[1, 6, 4, 5, 4, 3, 4, 9, 3, 4, 12] >2010-05-08
[6, 4, 5, 4, 3, 4, 9, 3, 4, 12, 2] 2010-05-08<


In [91]:
def collate_batch(data):
    batch = []
    for i in range(len(data[0])):
        batch_data = [torch.tensor(item[i], dtype=torch.int64) for item in data]
        batch_data = nn.utils.rnn.pad_sequence(batch_data, batch_first=True)
        batch.append(batch_data)


    return tuple(batch)

In [92]:
train_loader = DataLoader(dataset=train_dataset, collate_fn=collate_batch, batch_size = 64, num_workers = 8)
val_loader = DataLoader(dataset=val_dataset, collate_fn=collate_batch, batch_size = 64, num_workers = 8)

In [93]:
class EncoderGRU(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers=1, bidirectional=False):
        super(EncoderGRU, self).__init__()
        self.D = 2 if bidirectional else 1
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.gru = nn.GRU(
            vocab_size,
            hidden_size,
            num_layers=num_layers,
            bidirectional=bidirectional,
            batch_first=True,
        )

    def forward(self, x, hidden=None):
        if hidden == None:
            hidden = self.init_hidden(x.shape[0]).to(x.device)

        one_hot = F.one_hot(x, num_classes=self.vocab_size).float().to(x.device)

        output, hidden = self.gru(one_hot, hidden)

        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(
            self.D * self.gru.num_layers,
            batch_size,
            self.hidden_size,
            dtype=torch.float32,
        )

In [94]:
x_enc_batch,  x_dec_batch, y_batch = next(iter(train_loader))
print(x_enc_batch)
print(x_dec_batch)
print(y_batch)
encoder = EncoderGRU(len(human_vocab), hidden_size=32, num_layers=1, bidirectional=False)
print(encoder)
enc_out, enc_hidden = encoder(x_enc_batch)
print(enc_out.shape, enc_hidden.shape)

tensor([[10,  4, 13,  4,  6,  5],
        [14,  4,  8,  4, 12,  7],
        [10,  4,  9,  4, 13,  5],
        [ 8,  4, 12,  4, 13, 10],
        [13,  4,  9,  4, 13,  5],
        [10,  4,  9,  4,  5,  5],
        [10,  4,  9,  4,  5, 12],
        [10,  4,  6,  4, 12, 13],
        [ 7,  4,  6,  4, 12,  7],
        [ 8,  4, 11,  4,  5, 12],
        [13,  4, 12,  4,  6,  6],
        [ 8,  4, 12,  4,  6,  9],
        [ 7,  4,  8,  4, 13, 10],
        [10,  4, 12,  4, 13,  8],
        [11,  4, 11,  4, 14,  7],
        [ 8,  4,  9,  4, 14,  9],
        [12,  4,  8,  4, 12,  9],
        [ 6,  4, 10,  4, 12, 13],
        [ 6,  4, 10,  4, 12, 14],
        [14,  4, 10,  4,  5, 12],
        [10,  4, 12,  4,  6,  8],
        [ 7,  4,  9,  4,  5, 12],
        [ 6,  4, 12,  4,  5,  6],
        [10,  4,  6,  4, 12,  5],
        [ 9,  4, 13,  4,  5, 11],
        [ 8,  4,  8,  4, 12,  5],
        [10,  4,  8,  4, 14, 14],
        [10,  4,  9,  4,  7,  6],
        [ 7,  4,  8,  4, 13, 14],
        [ 9,  

In [95]:
lang = train_dataset.input_lang
for row in x_enc_batch:
    print(lang.ind_to_str(row))

5/8/10
9/3/72
5/4/80
3/7/85
8/4/80
5/4/00
5/4/07
5/1/78
2/1/72
3/6/07
8/7/11
3/7/14
2/3/85
5/7/83
6/6/92
3/4/94
7/3/74
1/5/78
1/5/79
9/5/07
5/7/13
2/4/07
1/7/01
5/1/70
4/8/06
3/3/70
5/3/99
5/4/21
2/3/89
4/2/04
7/1/21
3/2/08
7/3/72
7/6/22
3/4/17
3/9/76
3/2/17
7/6/79
3/4/94
9/6/85
8/5/84
8/1/95
8/8/86
6/5/98
5/1/11
1/4/21
1/5/74
9/5/89
7/7/79
8/1/98
4/1/15
8/4/16
9/6/83
7/6/04
4/5/73
3/3/82
5/6/00
5/3/80
6/2/19
8/1/80
6/3/82
8/5/76
9/7/17
1/5/11


In [96]:
class DecoderGRU(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers=1):
        super(DecoderGRU, self).__init__()
        self.hidden_size = hidden_size

        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.gru = nn.GRU(
            vocab_size, hidden_size, num_layers=num_layers, batch_first=True
        )
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden=None):
        if hidden == None:
            hidden = self.init_hidden(x.shape[0]).to(x.device)
        one_hot = F.one_hot(x, num_classes=self.vocab_size).float().to(x.device)
        output, hidden = self.gru(one_hot, hidden)
        output = self.fc(output)
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(
            self.gru.num_layers, batch_size, self.hidden_size, dtype=torch.float32
        )

In [97]:
encoder = EncoderGRU(len(human_vocab), hidden_size=32, num_layers=1, bidirectional=False)
decoder = DecoderGRU(len(machine_vocab), hidden_size=32, num_layers=1)
print(decoder)
print(encoder)

DecoderGRU(
  (gru): GRU(14, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=14, bias=True)
)
EncoderGRU(
  (gru): GRU(37, 32, batch_first=True)
)


In [98]:
print("Decoder forward pass\n")

# Teacher forcing
print("Training with teacher forcing")
print(f"Input batch shape: {x_dec_batch.shape}")
dec_out, dec_hid = decoder(x_dec_batch, enc_hidden)
print(f"decoder output shape: {dec_out.shape}\ndecoder hn shape: {dec_hid.shape}")
# loss(dec_out, target)
print()
# Without teacher forcing
print("Training without teacher forcing")
dec_input = x_dec_batch[:,0:1]
print(f"Decoder 1st input shape: {dec_input.shape}")
dec_out, dec_hid = decoder(dec_input, enc_hidden)
print(f"decoder output shape: {dec_out.shape}\ndecoder hn shape: {dec_hid.shape}")
next_input = torch.argmax(dec_out, dim=-1)
print(f"decoder 2nd input shape: {next_input.shape}")


Decoder forward pass

Training with teacher forcing
Input batch shape: torch.Size([64, 11])
decoder output shape: torch.Size([64, 11, 14])
decoder hn shape: torch.Size([1, 64, 32])

Training without teacher forcing
Decoder 1st input shape: torch.Size([64, 1])
decoder output shape: torch.Size([64, 1, 14])
decoder hn shape: torch.Size([1, 64, 32])
decoder 2nd input shape: torch.Size([64, 1])


In [99]:
class Seq2sec(nn.Module):
    def __init__(self, encoder, decoder) -> None:
        super(Seq2sec, self).__init__()

        self.encoder = encoder
        self.decoder = decoder

    def forward(self, enc_input_batch, sos_index = 1, dec_input_batch = None, teacher_forcing = False, out_length = 1):
        encoder_output, encoder_hidden = self.encoder(enc_input_batch)
        batch_size = len(enc_input_batch)

        if teacher_forcing:
            decoder_output, _ = self.decoder(dec_input_batch, encoder_hidden)
            return decoder_output
        else:
            decoder_input = (torch.zeros(batch_size, 1, dtype=torch.int64) + sos_index).to(enc_input_batch.device)
            decoder_output = torch.empty(batch_size, out_length, self.decoder.vocab_size).to(enc_input_batch.device)

            hidden = encoder_hidden

            for i in range(out_length):
                decoder_output_i, hidden = self.decoder(decoder_input, hidden)
                decoder_output[:,i:i+1,:] = decoder_output_i
                decoder_input = torch.argmax(decoder_output_i, dim=-1)

            return decoder_output


In [100]:
model = Seq2sec(encoder, decoder)
model

Seq2sec(
  (encoder): EncoderGRU(
    (gru): GRU(37, 32, batch_first=True)
  )
  (decoder): DecoderGRU(
    (gru): GRU(14, 32, batch_first=True)
    (fc): Linear(in_features=32, out_features=14, bias=True)
  )
)

In [101]:
model(x_enc_batch, out_length = 20).shape

torch.Size([64, 20, 14])

In [102]:
model(x_enc_batch, dec_input_batch = x_dec_batch, teacher_forcing = True).shape

torch.Size([64, 11, 14])

In [103]:
print("Model forward pass (input -> encoder -> decoder -> output)\n")

# Teacher forcing
print("Training without teacher forcing (out_length = 20)")
print(f"Input batch shape: {x_enc_batch.shape}")
output =  model(x_enc_batch, out_length = 20)
print(f"Output shape: {output.shape}")

print()
# Without teacher forcing
print("Training with teacher forcing")
print(f"Encoder input batch shape: {x_enc_batch.shape}")
print(f"Decoder input batch shape: {x_dec_batch.shape}")
output = model(x_enc_batch, dec_input_batch = x_dec_batch, teacher_forcing = True)
print(f"Output shape: {output.shape}")

Model forward pass (input -> encoder -> decoder -> output)

Training without teacher forcing (out_length = 20)
Input batch shape: torch.Size([64, 6])
Output shape: torch.Size([64, 20, 14])

Training with teacher forcing
Encoder input batch shape: torch.Size([64, 6])
Decoder input batch shape: torch.Size([64, 11])
Output shape: torch.Size([64, 11, 14])


In [123]:
class Trainer:
    def __init__(
        self,
        model,
        train_dataLoader,
        loss_fn,
        optimizer,
        val_dataLoader=None,
        padding_index=0,
        sos_index=1,
        teacher_forcing_ratio=0.5,
        device=device,
    ) -> None:
        self.model = model.to(device)
        self.train_dataLoader = train_dataLoader
        self.val_dataLoader = val_dataLoader
        self.loss_fn = loss_fn
        self.padding_index = padding_index
        self.sos_index = sos_index
        self.teacher_forcing_ratio = teacher_forcing_ratio
        self.device = device
        self.optimizer = optimizer

    def loss_value(self, output, target):
        C = output.shape[-1]

        output_flat = output.view(-1, C)
        target_flat = target.view(-1)

        loss = self.loss_fn(output_flat, target_flat)

        return loss

    def train_batch(self, encoder_input, decoder_input, target):
        teacher_forcing = np.random.random() < self.teacher_forcing_ratio

        output = self.model(
            encoder_input,
            dec_input_batch=decoder_input,
            teacher_forcing=teacher_forcing,
            sos_index=self.sos_index,
            out_length=target.shape[1],
        )

        loss = self.loss_value(output, target)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss.item()

    def train_one_epoch(self):
        model.train()
        epoch_loss = 0

        epoch_loss = 0
        batch_losses = []

        i = 1

        for enc_input, dec_input, target in self.train_dataLoader:
            i += 1
            enc_input = enc_input.to(self.device)
            dec_input = dec_input.to(self.device)
            target = target.to(self.device)

            batch_loss = self.train_batch(
                encoder_input=enc_input, decoder_input=dec_input, target=target
            )

            epoch_loss += batch_loss * len(enc_input)
            batch_losses.append(batch_loss)

        size = len(self.train_dataLoader.dataset)

        return epoch_loss / size, batch_losses

    def validation(self):
        size = len(self.val_dataLoader.dataset)
        model.eval()

        test_loss = 0

        with torch.inference_mode():
            for enc_input, _, target in self.val_dataLoader:
                enc_input = enc_input.to(self.device)
                target = target.to(self.device)

                out = self.model(
                    enc_input,
                    teacher_forcing=False,
                    sos_index=self.sos_index,
                    out_length=target.shape[1],
                )
                test_loss += self.loss_value(out, target).item() * len(enc_input)

            return test_loss / size

    def train(self, n_epochs, verbose = True):
        for epoch in range(n_epochs):
            epoch_loss, batch_losses = self.train_one_epoch()

            if self.val_dataLoader != None:
                val_loss = self.validation()

                if verbose:
                    print(f"Epoch {epoch + 1 :< 4}  training loss: {epoch_loss:>8f} | validation loss: {val_loss:>8f}")\
                    
            else:
                if verbose:
                    print(f"Epoch {epoch + 1 :< 10}  training loss: {epoch_loss:>8f}")

In [126]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
cross_entropy_loss = nn.CrossEntropyLoss(ignore_index=0)

trainer = Trainer(model, train_dataLoader=train_loader, val_dataLoader=val_loader, loss_fn= cross_entropy_loss, optimizer=optimizer)

In [127]:
trainer.train(40)

Epoch  1    training loss: 0.678488 | validation loss: 0.745853
Epoch  2    training loss: 0.667945 | validation loss: 0.731366
Epoch  3    training loss: 0.657139 | validation loss: 0.720331
Epoch  4    training loss: 0.649183 | validation loss: 0.715878
Epoch  5    training loss: 0.633723 | validation loss: 0.696993
Epoch  6    training loss: 0.634708 | validation loss: 0.689045
Epoch  7    training loss: 0.623650 | validation loss: 0.684317
Epoch  8    training loss: 0.619833 | validation loss: 0.673395
Epoch  9    training loss: 0.615610 | validation loss: 0.664424
Epoch  10   training loss: 0.607913 | validation loss: 0.662807
Epoch  11   training loss: 0.608215 | validation loss: 0.656183
Epoch  12   training loss: 0.600462 | validation loss: 0.667898
Epoch  13   training loss: 0.610181 | validation loss: 0.654832
Epoch  14   training loss: 0.599423 | validation loss: 0.646188
Epoch  15   training loss: 0.598755 | validation loss: 0.646291
Epoch  16   training loss: 0.597133 | va